# Step 2: Setup Data Splits

Before iterating on annotations, you need proper data splits. Without them, you'll contaminate your evaluation and build a model that only looks good on paper.

This step uses the **quickstart-groups** dataset (KITTI multimodal data with left/right cameras and point clouds) and creates:

- **Test set (15%)** - Frozen. Never used for selection or training. Final evaluation only.
- **Validation set (15%)** - For iteration decisions. Used to evaluate between training rounds.
- **Golden QA set (5%)** - Small, heavily reviewed. Detects label drift.
- **Pool (65%)** - Active learning pool. All new labels come from here.

> **Critical:** Splits are created at the **group level** (scene), not sample level. This ensures the same scene stays together across all slices (left, right, pcd), preventing data leakage.

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import random

DATASET_NAME = "annotation_tutorial"

## Load or Create the Dataset

We clone `quickstart-groups` to a persistent working dataset. This keeps your annotations separate from the zoo dataset.

In [ ]:
# Load or create the dataset (idempotent - safe to rerun)
if DATASET_NAME in fo.list_datasets():
    print(f"Loading existing dataset: {DATASET_NAME}")
    dataset = fo.load_dataset(DATASET_NAME)
    
    # Check if splits already exist
    existing_views = dataset.list_saved_views()
    if "pool" in existing_views:
        print("Splits already exist. Skipping creation.")
        SPLITS_EXIST = True
    else:
        SPLITS_EXIST = False
else:
    print(f"Creating new dataset: {DATASET_NAME}")
    source = foz.load_zoo_dataset("quickstart-groups")
    dataset = source.clone(DATASET_NAME)
    dataset.persistent = True
    SPLITS_EXIST = False

print(f"\nDataset: {dataset.name}")
print(f"Media type: {dataset.media_type}")
print(f"Group slices: {dataset.group_slices}")
print(f"Default slice: {dataset.default_group_slice}")
print(f"Num groups (scenes): {len(dataset.distinct('group.id'))}")
print(f"Num samples (all slices): {dataset.count()}")

## Understand the Grouped Structure

The `quickstart-groups` dataset is a **grouped dataset** from KITTI:

| Slice | Content | Purpose |
|-------|---------|--------|
| `left` | Left camera images | 2D detection annotation |
| `right` | Right camera images | Stereo pair (optional use) |
| `pcd` | Point cloud data | 3D cuboid annotation |

Each **group** represents one scene/frame with synchronized data across all sensors.

In [ ]:
# Explore a single group
group_ids = dataset.distinct("group.id")
example_group_id = group_ids[0]

print(f"Example group ID: {example_group_id}")
print(f"\nSamples in this group:")

for sample in dataset.match({"group.id": example_group_id}):
    slice_name = sample.group.name
    print(f"  {slice_name}: {sample.filepath.split('/')[-1]}")

## Create Splits at the Group Level

**Why group-level splits?**

If we split at the sample level, the same scene could end up in both train and test (just different slices). This causes data leakage - the model "sees" scenes at training time that appear in evaluation.

By splitting at the **group level**, we ensure:
- All slices from the same scene stay together
- No information leaks between splits

In [ ]:
# Create splits (only if they don't exist)
if not SPLITS_EXIST:
    random.seed(42)
    
    # Get unique group IDs (scenes)
    group_ids = dataset.distinct("group.id")
    random.shuffle(group_ids)
    
    n = len(group_ids)
    n_test = int(0.15 * n)      # 15% frozen test
    n_val = int(0.15 * n)       # 15% validation
    n_golden = int(0.05 * n)    # 5% golden QA
    # Remainder is pool
    
    test_groups = group_ids[:n_test]
    val_groups = group_ids[n_test:n_test + n_val]
    golden_groups = group_ids[n_test + n_val:n_test + n_val + n_golden]
    pool_groups = group_ids[n_test + n_val + n_golden:]
    
    print(f"Creating splits at GROUP level:")
    print(f"  Test:   {len(test_groups)} groups ({100*len(test_groups)/n:.0f}%)")
    print(f"  Val:    {len(val_groups)} groups ({100*len(val_groups)/n:.0f}%)")
    print(f"  Golden: {len(golden_groups)} groups ({100*len(golden_groups)/n:.0f}%)")
    print(f"  Pool:   {len(pool_groups)} groups ({100*len(pool_groups)/n:.0f}%)")

In [ ]:
# Tag ALL samples in each group with the split tag
if not SPLITS_EXIST:
    from fiftyone import ViewField as F
    
    # Tag by group membership (tags all slices in each group)
    dataset.match(F("group.id").is_in(test_groups)).tag_samples("split:test")
    dataset.match(F("group.id").is_in(val_groups)).tag_samples("split:val")
    dataset.match(F("group.id").is_in(golden_groups)).tag_samples("split:golden")
    dataset.match(F("group.id").is_in(pool_groups)).tag_samples("split:pool")
    
    # Save views for easy access
    dataset.save_view("test_set", dataset.match_tags("split:test"))
    dataset.save_view("val_set", dataset.match_tags("split:val"))
    dataset.save_view("golden_qa", dataset.match_tags("split:golden"))
    dataset.save_view("pool", dataset.match_tags("split:pool"))
    
    print("\nSplits created and saved as views.")
else:
    print("Using existing splits.")

In [ ]:
# Add annotation tracking field (idempotent)
if "annotation_status" not in dataset.get_field_schema():
    dataset.add_sample_field("annotation_status", fo.StringField)
    dataset.set_values("annotation_status", ["unlabeled"] * dataset.count())
    print("Added annotation_status field (all samples start as 'unlabeled')")
else:
    print("annotation_status field already exists.")

## Why These Splits Matter

| Split | Purpose | Rules |
|-------|---------|-------|
| **Test** | Final evaluation | Never touch. Never use for selection. Check only at the end. |
| **Val** | Iteration decisions | Evaluate after each training round. Guides what to label next. |
| **Golden** | Label quality check | Heavily reviewed. Re-check after each annotation round for drift. |
| **Pool** | Labeling source | All new annotations come from here. |

**If you skip this:** Your metrics become meaningless. You'll overfit to your own selection strategy.

In [ ]:
# Verify setup
from fiftyone import ViewField as F

print("Saved views:", dataset.list_saved_views())
print()

for view_name in ["test_set", "val_set", "golden_qa", "pool"]:
    view = dataset.load_saved_view(view_name)
    # Count unique groups in view
    n_groups = len(view.distinct("group.id"))
    n_samples = len(view)
    print(f"{view_name}: {n_groups} groups, {n_samples} samples (all slices)")

In [ ]:
# Verify no group ID appears in multiple splits
test_groups = set(dataset.load_saved_view("test_set").distinct("group.id"))
val_groups = set(dataset.load_saved_view("val_set").distinct("group.id"))
pool_groups = set(dataset.load_saved_view("pool").distinct("group.id"))

assert len(test_groups & val_groups) == 0, "Leak between test and val!"
assert len(test_groups & pool_groups) == 0, "Leak between test and pool!"
assert len(val_groups & pool_groups) == 0, "Leak between val and pool!"

print("Split integrity verified: no group appears in multiple splits.")

## Launch the App

Explore your grouped dataset in the App. Notice:
- The **group mode** shows synchronized samples
- Use the **slice selector** to switch between left, right, and pcd
- Filter by split tags to see each partition

In [ ]:
session = fo.launch_app(dataset)

## Summary

You created four data splits with clear purposes:
- Test (frozen), Val (iteration), Golden (QA), Pool (labeling source)
- **Splits are at the group level** - same scene = same split across all slices

**Artifacts:**
- `annotation_tutorial` dataset (persistent clone of quickstart-groups)
- Split tags: `split:test`, `split:val`, `split:golden`, `split:pool`
- Saved views: `test_set`, `val_set`, `golden_qa`, `pool`
- `annotation_status` field for tracking progress

**Next:** Step 3 - Smart Sample Selection